In [26]:
import sys
sys.path.append('../../../')

import pandas as pd

import keras

from utils.predictor import PredictorToolkit
from utils.dataset import TreadingDataset
from utils.strategy import TradeStrategy
from utils.backtester import Backtester
from utils.field_processing import create_ts_from_df
import warnings

warnings.filterwarnings('ignore')

In [2]:
SEQUENCE_LENGTH = 9
SHIFT_SIZE = 3

# Loading the model

In [13]:
model = keras.models.load_model('../../../models/test/best_model_EURUSD.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 512)            │     1,099,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,311,669 (12.63 MB)

 Trainable params: 1,103,889 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,207,780 (8.42 MB)

# Loading the previous sequences

In [14]:
base_df = pd.read_csv('../../../btc_last_test.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
# base_df = pd.read_csv('../../../EURUSD_H1_202403_2025.csv', sep='\t', usecols=['<DATE>', '<TIME>', '<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>'])
base_df.rename(columns={
    '<OPEN>': 'OPEN', 
    '<HIGH>': 'HIGH', 
    '<LOW>': 'LOW', 
    '<CLOSE>': 'CLOSE'
}, inplace=True)

base_df = create_ts_from_df(base_df, '<DATE>', '<TIME>', 'DATETIME')
base_df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-03-07 19:00:00,1.08589,1.08644,1.08319,1.08434
2025-03-07 20:00:00,1.08432,1.08442,1.08256,1.08398
2025-03-07 21:00:00,1.08397,1.08440,1.08333,1.08425
2025-03-07 22:00:00,1.08424,1.08492,1.08399,1.08462
2025-03-07 23:00:00,1.08461,1.08506,1.08320,1.08325


In [18]:
df = base_df.loc[:'2025-03-09 15:00:00']
df.tail()

,OPEN,HIGH,LOW,CLOSE
DATETIME,,,,
2025-03-07 19:00:00,1.08589,1.08644,1.08319,1.08434
2025-03-07 20:00:00,1.08432,1.08442,1.08256,1.08398
2025-03-07 21:00:00,1.08397,1.08440,1.08333,1.08425
2025-03-07 22:00:00,1.08424,1.08492,1.08399,1.08462
2025-03-07 23:00:00,1.08461,1.08506,1.08320,1.08325


# Generate the next close price 

In [24]:
dataset = TreadingDataset(df[['CLOSE']])
dataset.prepare_data()

In [25]:
last_prev_lines = dataset.feature_sequence[-1]
tk = PredictorToolkit(dataset.target_scaler, model, SEQUENCE_LENGTH)

for _ in range(1):
    last_prev_lines = dataset.feature_sequence[-1]
    entry_price = dataset.df[['CLOSE']].iloc[-1:].values[0][0]
    new_df = tk.predict_the_next_price(dataset.df[['CLOSE']], entry_price, last_prev_lines)
    dataset.update_df(new_df)

dataset.calculate_atr(base_df, period=SEQUENCE_LENGTH)
tk.predict_df

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


,ENTRY_PRICE,PREDICTED_CLOSE
2025-03-08,1.08325,1.083516


# Make the trade strategy

## Stratégie avec un ratio perte/gain (Ratio Risk/Reward) de 1:2

perte de 10 pips pour gagner 20 pips
- Stop-loss pips = 0.0010
- Take-profit pips (Stop-loss pips * 2) = 0.0020


In [21]:
dataset.df.tail()

,CLOSE,EMA,RSI,MACD,SIGNAL_LINE,UPPER_BAND,LOWER_BAND,BOLLINGER_PERCENT_B,ATR
2025-03-07 19:00:00,1.08434,1.083537,63.876652,0.001790,0.001785,1.088663,1.077196,0.622998,0.003013
2025-03-07 20:00:00,1.08398,1.083580,60.150943,0.001611,0.001750,1.088624,1.077728,0.573784,0.002918
2025-03-07 21:00:00,1.08425,1.083643,60.888889,0.001474,0.001695,1.088499,1.078403,0.579136,0.002926
2025-03-07 22:00:00,1.08462,1.083736,58.910506,0.001380,0.001632,1.088297,1.079194,0.596069,0.002853
2025-03-07 23:00:00,1.08325,1.083690,51.066961,0.001180,0.001542,1.087942,1.079972,0.411285,0.002878


In [22]:
strategy = TradeStrategy(tk.predict_df, 2)
strategy.run(dataset.df)
strategy.df

,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT


# Backtesting du model

In [10]:
backtest_data = base_df[['CLOSE']].merge(
    strategy.df, left_index=True, right_index=True
    ).merge(dataset.df[['ATR']], left_index=True, right_index=True)
backtest_data.merge(base_df[['HIGH', 'LOW']], left_index=True, right_index=True, how='inner')

,CLOSE,ENTRY_PRICE,PREDICTED_CLOSE,POSITION,STOP_LOSS,TAKE_PROFIT,ATR,HIGH,LOW
2025-03-06 16:00:00,1.08373,1.08263,1.080028,SELL,1.084908,1.07222,0.002711,1.08411,1.08113


In [11]:
backtester = Backtester(backtest_data)
backtester.simulate_trades()
performance_metrics = backtester.calculate_performance_metrics()

print("Métriques de performance :")
for key, value in performance_metrics.items():
    print(f"{key}: {value}")

Métriques de performance :
total_trades: 1
winning_trades: 0
losing_trades: 1
win_rate: 0.0
net_profit: -0.001100000000000101
average_profit: -0.001100000000000101
sharpe_ratio: nan


In [12]:
# Test de différents multiplicateurs
multipliers = [(1.0, 1.5), (1.5, 2.0), (2.0, 2.5)]

best_metrics = None
best_multipliers = None

for stop_loss_multiplier, take_profit_multiplier in multipliers:
    # Appliquez les multiplicateurs et recalculez les stop-loss et take-profit
    backtest_data['STOP_LOSS'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] + (stop_loss_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] - (stop_loss_multiplier * row['ATR']),
        axis=1
    )
    backtest_data['TAKE_PROFIT'] = backtest_data.apply(
        lambda row: row["PREDICTED_CLOSE"] - (take_profit_multiplier * row['ATR'])
            if row["POSITION"] == "SELL"
            else row["PREDICTED_CLOSE"] + (take_profit_multiplier * row['ATR']),
        axis=1
    )

    # Simule les trades et calcule les métriques
    backtester = Backtester(backtest_data)
    backtester.simulate_trades()
    metrics = backtester.calculate_performance_metrics()

    # Compare les métriques pour trouver les meilleurs paramètres
    if best_metrics is None or metrics['net_profit'] > best_metrics['net_profit']:
        best_metrics = metrics
        best_multipliers = (stop_loss_multiplier, take_profit_multiplier)

print("Meilleurs multiplicateurs :", best_multipliers)
print("Métriques associées :", best_metrics)

Meilleurs multiplicateurs : (1.0, 1.5)
Métriques associées : {'total_trades': 1, 'winning_trades': 0, 'losing_trades': 1, 'win_rate': 0.0, 'net_profit': -0.00010904900021024844, 'average_profit': -0.00010904900021024844, 'sharpe_ratio': nan}
